# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
    - Membre 1
    - Membre 2
    - Membre 3




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [9]:
import numpy as np
import math
import copy
from collections import deque

class State:
    
    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)
        
        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None
        
        self.nb_moves = 0
        self.score = 0
        
        # TODO
        self.rock = None
        
        self.debug = []

    """
    Constructeur d'un état à partir du mouvement (c,d)
    """
    def move(self, c, d):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        s.rock = self.rock
        # TODO
        return s

    def put_rock(self, rock_pos):
        s = State(self.pos)
        s.prev = self
        s.c = self.c
        s.d = self.d
        s.nb_moves = self.nb_moves
        s.rock = rock_pos
        # TODO
        return s
            
    def score_state(self, rh):
        
        sc_mat = self.score_matrix()
        
        move_on = rh.move_on
        
        score = 0
        
        
    
        # calcul des moyenne pour chaque voiture
        for i in range(1, len(move_on)):
            if rh.horiz[i]:
                score_car = self.get_car_score(sc_mat, move_on[i], self.pos[i], rh.length[i], rh.horiz[i])
            else:
                score_car = self.get_car_score(sc_mat, self.pos[i], move_on[i], rh.length[i], rh.horiz[i])
            score += score_car
            self.debug.append([rh.color[i], score_car])
        
        pos_red = self.pos[0]
        move_on = rh.move_on # which row or column the car is moving on
        # basically we want to check if there any car on the right part of the line where the car is present
        # that is the line with the index no. 2
        nb_cars_total = 0 # the previously said number of the cars
        nb_rock = 0
        bad_cars = []
        bad_move = 0
        for i in range(1, len(move_on)): # skipping the first car (the red one)
            if not rh.horiz[i]:
                if (move_on[i] >= (self.pos[0] + rh.length[0])) and ((self.pos[i] <= 2) and (self.pos[i] + rh.length[i] - 1 >= 2)):
                    nb_cars_total += 1
                    bad_cars.append(i) # storing the cars in front of the red car
                    for j in [x for x in range(1, len(move_on)) if x != i]: # iterating the cars except the red and the current one
                        if rh.horiz[j]: # si la voiture bouge horizontalement
                            if (move_on[i] >= self.pos[j]) and (move_on[i] <= self.pos[j] + rh.length[j] - 1):
                                nb_cars_total += 1
                        else:
                            if (move_on[j] == move_on[i]): # the car move on the same column (not really likely)
                                nb_cars_total += 1
                    if self.rock != None:
                        if self.rock[1] == move_on[i] and (self.rock[0] == self.pos[i] - 1 or self.rock[0] == self.pos[i] + 1):
                            nb_rock = 1 # un seul rock possible
        
        if self.c not in bad_cars: # this is a "bad move" because we are moving a car which is not in front of the red car
                                   # we should prioritize moving the cars blocking the red one
            bad_move = 2
            
        if self.rock != None:
            if self.rock[0] == 2 and self.rock[1] > self.pos[0]:
                nb_rock = 2
            
        score += pos_red * 2 - nb_cars_total - nb_rock - bad_move
        
        
        
        return score
            
    def get_car_score(self, mat, index_ligne, index_col, length, horiz):
        
        score_car = 0
        for i in range(length):
            score_car += float(mat[index_ligne][index_col + i]) if horiz else float(mat[index_ligne + i][index_col])
        
        return float(score_car)
    
    def score_state_prev(self, rh):
        """
        Pour evaluer le score on procede comme suit :
         - On prend la position de la voiture rouge comme score de depart, multiplié par
           deux pour ajouter de l'importance au fait de deplacer la voiture
         - Si une voiture est devant la voiture rouge, on enleve 1 au score
         - Si un rocher est devant la voiture rouge, on enleve 1
         - Si un rocher est devant une des voitures qui bloque la voiture rouge (juste devant)
           on enleve 1
         - Si une voiture bloque une des voitures qui est devant la voiture rouge
           on enleve 1
        Ainsi le pire score possible est quand il y a 
        une voiture devant toute les cases possibles de la voiture rouge, cad 4, et un rocher
        devant une de ces voitures, donc -5.
        Le meilleur score est 4.
        """
        pos_red = self.pos[0]
        move_on = rh.move_on # which row or column the car is moving on
        # basically we want to check if there any car on the right part of the line where the car is present
        # that is the line with the index no. 2
        nb_cars_total = 0 # the previously said number of the cars
        nb_rock = 0
        bad_cars = []
        bad_move = 0
        for i in range(1, len(move_on)): # skipping the first car (the red one)
            if not rh.horiz[i]:
                if (move_on[i] >= (self.pos[0] + rh.length[0])) and ((self.pos[i] <= 2) and (self.pos[i] + rh.length[i] - 1 >= 2)):
                    nb_cars_total += 1
                    bad_cars.append(i) # storing the cars in front of the red car
                    for j in [x for x in range(1, len(move_on)) if x != i]: # iterating the cars except the red and the current one
                        if rh.horiz[j]: # si la voiture bouge horizontalement
                            if (move_on[i] >= self.pos[j]) and (move_on[i] <= self.pos[j] + rh.length[j] - 1):
                                nb_cars_total += 1
                        else:
                            if (move_on[j] == move_on[i]): # the car move on the same column (not really likely)
                                nb_cars_total += 1
                    if self.rock != None:
                        if self.rock[1] == move_on[i] and (self.rock[0] == self.pos[i] - 1 or self.rock[0] == self.pos[i] + 1):
                            nb_rock = 1 # un seul rock possible
        if self.c not in bad_cars: # this is a "bad move" because we are moving a car which is not in front of the red car
                                   # we should prioritize moving the cars blocking the red one
            bad_move = 1
            
        if self.rock != None:
            if self.rock[0] == 2 and self.rock[1] > self.pos[0]:
                nb_rock = 2
            
        self.score = (pos_red * 2 - nb_cars_total - nb_rock - bad_move)

    def score_matrix(self):
        mat = np.zeros((6,6))
        red_car_pos = self.pos[0] # position voiture rouge
        
        #nb_col_left = red_car_pos + 2
        #nb_col_right = 6 - nb_col_left
        
        base = [-7,-8,-9,-8,-7,-6]
        
        # construction de la partie gauche de la matrice a partir du devant de la voiture rouge
        #  generation des score de la colonne devant la voiture rouge
        if red_car_pos <= 3:
            mat[:,red_car_pos+2] = base
        #  generation de la partie gauche
        for i in range(6):
            temp = base[i] + 1
            for j in range(red_car_pos + 1,-1,-1):
                mat[i,j] = temp
                temp += 1
        # construction de la partie droite, egale a la colonne devant la voiture rouge
        if red_car_pos <= 2:
            for i in range(red_car_pos + 3, 6):
                for j in range(6):
                    base[j] = base[j] - 1
                mat[:,i] = base
                
        return mat

    def success(self):
        return self.pos[0] == 4
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")
        
        return np.array_equal(self.pos, other.pos)
    
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.score) < (other.score)


La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [4]:
class Rushhour:
    
    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color
        
        self.free_pos = None
    
    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i]+self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i]+self.length[i], self.move_on[i]] = False
        if state.rock != None:
            self.free_pos[state.rock] = False
        # TODO
    
    def possible_moves_tmp(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[self.move_on[i], state.pos[i]+self.length[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1))
            else:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[state.pos[i]+self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1))
        return new_states
    
    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(len(state.pos)):
            curr_pos = state.pos[i]
            if self.horiz[i]: # car moves horizontally
                curr = self.free_pos[self.move_on[i]] # the current line
            else: # car moves vertically
                curr = self.free_pos[:,self.move_on[i]] # the current column (but represented as a line in curr)
            if curr_pos + self.length[i] < 6: # not on the far right
                if curr[curr_pos + self.length[i]]: # free slot on the right
                        new_state = state.move(i, 1)
                        new_states.append(new_state)
            if curr_pos > 0: # not on the far left
                if curr[curr_pos-1]: # free slot on the left
                    new_state = state.move(i, -1)
                    new_states.append(new_state)
                        
        return new_states
    
    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states =[]
        if state.rock != None :
            for i in range(len(self.free_pos[0])):
                for j in range(len(self.free_pos[1])):
                    if i != 2 and i != state.rock[0] and j != state.rock[1] and self.free_pos[i][j]:
                        new_states.append(state.put_rock((i, j)))
        else:
            for i in range(len(self.free_pos[0])):
                for j in range(len(self.free_pos[1])):
                    if i != 2 and self.free_pos[i][j]:
                        new_states.append(state.put_rock((i, j)))
        #TODO
        return new_states
    

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid= np.chararray((6, 6))
        grid[:]='-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] +self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock != None :
            grid[state.rock] = 'x'
        print(grid)

### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [7]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [8]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversarielle avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [28]:
horiz_dic = {-1: "la gauche",
             1: "la droite"}
verti_dic = {-1: "le haut",
             1: "le bas"}

import random

class MiniMaxSearch:
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth
        self.visited_states = set()

    def minimax_1(self, current_depth, current_state): 
        #TODO
        if current_depth == 0 or current_state.success():
            current_state.score_state(self.rushhour)
            return current_state
        value_best = -(np.Infinity)
        states = []
        possiblemoves = self.rushhour.possible_moves(current_state)
        for s in possiblemoves:
            curr_st = self.minimax_1(current_depth - 1, s)
            value = curr_st.score
            if value == value_best:
                states.append(curr_st)
            elif value_best < value:
                states = [curr_st]
                value_best = value
        best_move = random.choice(states)
        return best_move
    
    def minimax_1_tmp(self, current_depth, current_state): 
        #TODO
        if current_depth == 0 or current_state.success():
            return current_state
        value_best = -(np.Infinity)
        best_move = current_state
        for s in self.rushhour.possible_moves(current_state):
            curr_st = self.minimax_1(current_depth - 1, s)
            value = curr_st.score_state(self.rushhour)
            visited = False
            for st in self.visited_states:
                visited = visited or (st == curr_st)
            if value_best < value and not visited:
                best_move = s
                value_best = value
        self.print_move(True, best_move)
        print(best_move.pos[0])
        return best_move
    
    def minimax_2(self, current_depth, current_state, is_max): 
        if current_depth == 0 or current_state.success():
            return current_state
        best_move = current_state
        if is_max:
            value_best = -(np.Infinity)
            for s in self.rushhour.possible_moves(current_state):
                value = self.minimax_2(current_depth - 1, s, False).score_state(self.rushhour)
                if value_best < value:
                    best_move = s
                    value_best = value
        else:
            value_best = np.Infinity
            for s in self.rushhour.possible_moves(current_state):
                value = self.minimax_2(current_depth - 1, s, True).score_state(self.rushhour)
                if value_best > value:
                    best_move = s
                    value_best = value
        return best_move

    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta):
        #TODO
        return best_move

    def expectimax(self, current_depth, current_state, is_max):
        #TODO
        return best_move

    def decide_best_move_1(self):
        self.state = self.minimax_1(self.search_depth, self.state)
        return 0

    def decide_best_move_2(self, is_max):
        self.state = self.minimax_2(self.search_depth, self.state, is_max)
        return 0

    def decide_best_move_pruning(self, is_max):
        # TODO
        return 0

    def decide_best_move_expectimax(self, is_max):
        # TODO
        return 0

    def solve(self, state, is_singleplayer):
        
        total_moves = 0
        
        while not self.state.success():
            total_moves += 1
            self.decide_best_move_1()
            self.print_move(True, self.state)
            
        #else:
        #    self.solve(self.decide_best_move_2(), is_singleplayer)
        print("Total moves :", total_moves)

    def print_move(self, is_max, state):
        if is_max:
            dir = horiz_dic[state.d] if self.rushhour.horiz[state.c] else verti_dic[state.d]
            print("Voiture " + self.rushhour.color[state.c] + " vers " + dir)
        else:
            print("Roche a la position ",str(state.rock))
        return 0
          


### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [4]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

Voiture rouge vers la droite
Roche a la position  (3, 1)


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [32]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3) 
algo.rushhour.init_positions(s)
algo.rushhour.print_pretty_grid(s)
algo.solve(s, True)
%time

[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture jaune vers la gauche
[[b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture bleu vers le haut
[[b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'r' b'r' b'-' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
Voiture jaune vers la droite
[[b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'r' b'r' b'-' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'j' b'j' b'j']]
Voiture vert vers le haut
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'r' b'r' b'-' b'-' b'v' b

Voiture jaune vers la droite
[[b'-' b'o' b'-' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'b']
 [b'-' b'o' b'r' b'r' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'j' b'j' b'j']]
Voiture jaune vers la gauche
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'o' b'r' b'r' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'j' b'j' b'j' b'-']]
Voiture vert vers le bas
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'b']
 [b'-' b'o' b'r' b'r' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'j' b'j' b'j']]
Voiture orange vers le haut
[[b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'v' b'b']
 [b'-' b'o' b'r' b'r' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'j' b'j' b'j']]
Voiture vert vers le bas
[[b'-' b'o' b'-' b'-' b'-' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'b']
 [

In [19]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)
algo.rushhour.print_pretty_grid(s)
algo.solve(s, True) 
%time

[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture bleu vers le bas
Voiture jaune vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture bleu vers le bas
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture bleu vers le haut
Voiture jaune vers le bas
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture bleu vers le bas
Voiture jaune vers le haut

Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture bleu vers le bas
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture bleu vers le bas
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture bleu vers le haut
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le bas
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture bleu vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture bleu vers le bas
Voiture 

Voiture bleu vers le bas
Voiture jaune vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture jaune vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture bleu vers le 

Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture bleu vers le bas
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture bleu vers le haut
Voiture jaune vers le bas
Voiture bleu vers le bas
Voiture jaune vers le haut
Voiture bleu vers le bas
Voiture jaune vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture bleu vers le bas
Voiture jaune vers le haut
Voiture bleu vers le haut
Voiture bleu vers le haut
Voiture jaune vers le bas
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune vers le haut
Voiture bleu vers le bas
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture bleu vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture jaune vers le haut
Voiture jaune vers le bas
Voiture bleu vers le haut
Voiture bleu vers le bas
Voiture bleu vers le haut
Voiture jaune

RecursionError: maximum recursion depth exceeded while calling a Python object

In [62]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
[-9, -9, -10, -10, -11, -10, -10]
Voiture 2 vers la gauche
[-8, -10, -8, -9, -9, -9, -10, -9, -9]
Voiture rouge vers la droite
[-9, -8, -9, -7, -8, -8, -8, -9, -8, -8]
Voiture 2 vers la gauche
[-8, -7, -8, -7, -7, -7, -8, -7, -7]
Voiture 3 vers le haut
[-8, -7, -8, -4, -7, -7, -7, -7, -7]
Voiture 3 vers le haut
[-3, -5, -4, -5, -7, -4, -4, -5]
Voiture rouge vers la droite
[-4, -3, -4, -3, -3, -4]
Voiture 1 vers le haut
[-4, -3, -4, -3, -3, -3, -4]
Voiture 2 vers la gauche
[-4, -3, -3, -3, -4]
Voiture 4 vers le bas
[-4, -3, -3, 0, -3, -4]
Voiture 4 vers le bas
[1, -1, 0, -3, 0, -1]
Voiture rouge vers la droite
[0, 1, 1, 0]
Voiture 5 vers le bas
[0, 1, 1, 0]
Voiture 2 vers la droite
[0, 1, 1, 1, 0]
Voiture 5 vers le haut
[0, 1, 1, 1, 0]
Voiture 1 vers le ba

[-1, -3, -2, -2, -6, -1, -2]
Voiture rouge vers la droite
[-2, -1, -1, 0, -1]
Voiture 2 vers la droite
[-2, -1, -1, 0, -1]
Voiture 1 vers le bas
[-2, -1, -1, -1, 0, -1]
Voiture 8 vers le bas
[-2, -1, -1, -1, -1, -1, -1]
Voiture 1 vers le haut
[-2, -1, -1, -1, -1, -1]
Voiture 8 vers le bas
[-1, -1, -1, -1, -1]
Voiture 8 vers le bas
[-1, -1, 0, -1, -1]
Voiture 6 vers la gauche
[0, 0, 0, -1, 0, 0]
Voiture 6 vers la gauche
[0, 0, 0, 0, 0]
Voiture 7 vers la droite
[0, 0, 0, 0, 0, 0]
Voiture 1 vers le haut
[0, 0, 0, 0, 0]
Voiture 2 vers la gauche
[0, 0, 0, 0, 0]
Voiture 5 vers le haut
[0, 0, 0, 0, 0]
Voiture 1 vers le bas
[0, 0, 0, 0, 0, 0]
Voiture 2 vers la droite
[0, 0, 0, 0, 0, 0]
Voiture 6 vers la droite
[0, 0, 0, 0, 0, 0]
Voiture 1 vers le bas
[0, 0, 0, 0, 0, 0]
Voiture 6 vers la gauche
[0, 0, 0, 0, 0, 0]
Voiture 1 vers le bas
[0, 0, 0, 0, 0, 0, 0]
Voiture 2 vers la gauche
[0, 0, 0, 0, 0, 0]
Voiture 5 vers le bas
[0, 0, 0, 0, 0, 0]
Voiture 2 vers la droite
[0, 0, 0, 0, 0, 0, 0]
Voiture 

Voiture 1 vers le haut
[0, 0, 0, 0, 0, 0]
Voiture 2 vers la droite
[0, 0, 0, 0, 0, 0]
Voiture 7 vers la gauche
[0, 0, 0, 0, 0]
Voiture 5 vers le bas
[0, 0, 0, 0, 0]
Voiture 2 vers la gauche
[0, 0, 0, 0, 0]
Voiture 5 vers le haut
[0, 0, 0, 0, 0]
Voiture 1 vers le bas
[0, 0, 0, 0, 0, 0]
Voiture 1 vers le bas
[0, 0, 0, 0, 0, 0]
Voiture 6 vers la gauche
[0, 0, 0, 0, 0]
Voiture 3 vers le bas
[0, 0, 0, 0, 1]
Voiture 7 vers la droite
[0, 0, 0, 0, 0, 1]
Voiture 5 vers le bas
[0, 0, 0, 0, 0, 1]
Voiture 2 vers la droite
[0, 0, 0, 0, 0, 0, 1]
Voiture 7 vers la droite
[0, 0, 0, 0]
Voiture 1 vers le bas
[0, 0, 0]
Voiture 5 vers le haut
[0, 0, 0]
Voiture 1 vers le haut
[0, 0, 0, 0]
Voiture 2 vers la gauche
[0, 0, 0]
Voiture 5 vers le bas
[0, 0, 0]
Voiture 2 vers la droite
[0, 0, 0, 0]
Voiture 1 vers le bas
[0, 0, 0]
Voiture 7 vers la gauche
[0, 0, 0, 0, 0, 1]
Voiture 7 vers la gauche
[0, 0, 0, 0, 1]
Voiture 5 vers le haut
[0, 0, 0, 0, 1]
Voiture 3 vers le haut
[0, 0, 0, 0, 0]
Voiture 5 vers le bas
[

[-1, -3, -2, -2, -2, -6, -1, -2]
Voiture 8 vers le bas
[-1, -3, -2, -2, -2, -6, -2, -2]
Voiture 1 vers le bas
[-1, -3, -2, -2, -6, -2, -2]
Voiture 7 vers la droite
[-1, -3, -2, -2, -6, -2]
Voiture 1 vers le haut
[-1, -3, -2, -2, -2, -6, -2]
Voiture 2 vers la droite
[-1, -3, -2, -2, -2, -6, -2]
Voiture 7 vers la gauche
[-1, -3, -2, -2, -2, -6, -2, -2]
Voiture 1 vers le bas
[-1, -3, -2, -2, -2, -6, -2, -2]
Voiture 8 vers le haut
[-1, -3, -2, -2, -2, -6, -1, -2]
Voiture 2 vers la gauche
[-1, -3, -2, -2, -6, -1, -2]
Voiture rouge vers la gauche
[-2, -3, -3, -7, -2, -3]
Voiture 2 vers la droite
[-2, -3, -3, -3, -7, -2, -3]
Voiture 1 vers le bas
[-2, -4, -3, -7, -2, -3]
Voiture rouge vers la droite
[-1, -3, -2, -6, -1, -2]
Voiture 8 vers le bas
[-1, -3, -2, -6, -2, -2]
Voiture 7 vers la droite
[-1, -3, -2, -6, -2]
Voiture 1 vers le haut
[-1, -3, -2, -2, -2, -6, -2]
Voiture rouge vers la gauche
[-2, -3, -3, -3, -7, -3]
Voiture 1 vers le haut
[-2, -3, -3, -3, -7, -3]
Voiture 7 vers la gauche
[

### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [74]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]


TypeError: decide_best_move_2() missing 1 required positional argument: 'is_max'

In [0]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

In [0]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [0]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

In [0]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

In [0]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [0]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

In [0]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

In [0]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 